In [5]:
%matplotlib inline
import keras
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import scipy.ndimage
from keras.optimizers import SGD
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [2]:
train_labels=pd.read_csv('../../train.csv',engine='c')

In [10]:
trainGen=ImageDataGenerator(featurewise_center=False, 
                    samplewise_center=False, 
                     featurewise_std_normalization=False, 
                     samplewise_std_normalization=False, 
                     zca_whitening=False, 
                     zca_epsilon=1e-06, 
                     rotation_range=0, 
                     width_shift_range=10, 
                     height_shift_range=10, 
                     brightness_range=None, 
                     shear_range=0.0, 
                     zoom_range=0.0, 
                     channel_shift_range=0.1, 
                     fill_mode='nearest', 
                     cval=0.0, 
                     horizontal_flip=True, 
                     vertical_flip=False, 
                     rescale=None, 
                     preprocessing_function=None, 
                     data_format=None, 
                     validation_split=0.3)

train_generator = trainGen.flow_from_directory(
        '../../train',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

Found 25361 images belonging to 5005 classes.


# Build Model

In [ ]:
model = Sequential()
# input 224x224x3
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Conv2D(256,(3,3),activation='relu'))
model.add(Conv2D(256,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5005,activation='softmax'))

sgd=SGD(lr=0.001,decay=1e-6,momentum=0.9,nesterov=True) #??

model.compile(loss='categorical_crossentropy',optimizer=sgd) #??


In [ ]:
model.fit_generator(train_generator,steps_per_epoch=100,epochs=10)

Epoch 1/10


INFO:plaidml:Analyzing Ops: 179 of 389 operations complete
INFO:plaidml:Analyzing Ops: 198 of 389 operations complete


100/100 [==============================] - 153s 2s/step - loss: 7.5552

INFO:plaidml:Analyzing Ops: 179 of 389 operations complete
INFO:plaidml:Analyzing Ops: 204 of 389 operations complete


100/100 [==============================] - 150s 2s/step - loss: 7.6154